In [0]:
dbutils.unmount("/mnt/sales")

storage_account_name = "storage account name"
container_name = "container name"
mount_point = "mountpath"
access_key = "<access_key>"

# Mount
dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
  mount_point = mount_point,
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)


True

In [0]:
%fs
ls /mnt/sales

path,name,size,modificationTime
dbfs:/mnt/sales/discarded/,discarded/,0,0
dbfs:/mnt/sales/landing/,landing/,0,0
dbfs:/mnt/sales/stagging,stagging,224,1747501221000
dbfs:/mnt/sales/staging/,staging/,0,0


In [0]:
%fs
ls  /mnt/sales/landing

path,name,size,modificationTime
dbfs:/mnt/sales/landing/week24_orders.csv,week24_orders.csv,224,1747501366000


In [0]:
Orders_DF=spark.read.csv("file_path.csv", inferSchema=True, header=True)

In [0]:
Orders_DF.display()

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00Z,11599,CLOSED
2222222,2013-07-25T00:00:00Z,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00Z,12111,COMPLETE
4444444,2013-07-25T00:00:00Z,8827,CLOSED


In [0]:
errorFlg=False

OrdersCount=Orders_DF.count()
print(OrdersCount)

DistinctCount=Orders_DF.select('order_id').distinct().count()
print(DistinctCount)

if OrdersCount != DistinctCount:
    errorFlg = True

if errorFlg:
    dbutils.fs.mv("source", "destination")
    dbutils.notebook.exit('{"errorFlg":"True", "errorMsg":"Duplicate records found"}')

Orders_DF.createOrReplaceTempView("Orders")    

In [0]:
dbServer = "<your_db_server_here>"
dbName = 'database_name'
dbPort = '1433'
dbUser = 'datalogin'
dbPassword = 'sql-password'

In [0]:
# Define connection URL
connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(
    dbServer, dbPort, dbName, dbUser
)

# Get password from Databricks secret scope
dbPassword = dbutils.secrets.get(scope=databricksScope, key='sql-password')

# JDBC properties
connectionProperties = {
    'password': dbPassword,
    'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}


In [0]:
valid_Status_DF=spark.read.jdbc(url=connectionUrl, table="dbo.database_table_name", properties=connectionProperties)
display(valid_Status_DF)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPE CTED_FRAU D
COMPLETE
PENDING
CANCELED
PENDING_PAYMENT


In [0]:
valid_Status_DF.createOrReplaceTempView("valid_Status")

In [0]:
invalid_rows_df=spark.sql("select * from Orders where order_status not in (select status_name from valid_Status)")

In [0]:
display(invalid_rows_df)

order_id,order_date,customer_id,order_status


In [0]:
if invalid_rows_df.count()>0:
    errorFlg=True

if errorFlg:
    dbutils.fs.mv("source", "destination")
    dbutils.notebook.exit('{"errorFlg":"True", "errorMsg":"Invalid status found"}')
else:
    dbutils.fs.mv("source", "destination")
    dbutils.notebook.exit('{"errorFlg":"False", "errorMsg":"All good"}')